### Import

In [17]:
import os
import sys
import mlflow
# 🔁 Add your project root to Python path
project_root = os.path.abspath("..")  # or "../ProjectName" if nested deeper
if project_root not in sys.path:
    sys.path.insert(0, project_root)
# Now you can import modules from the root directory
import torch
import numpy as np
import matplotlib.pyplot as plt
import glob
from src.dataloader import get_dataloaders
from src.models.resnet import get_model  # or use build_effnet if swapping
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from src.dataloader import get_dataloaders
from src.models.resnet import get_model  # or adapt for EffNet later


## Working

In [13]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:

# mlflow.set_tracking_uri("file:../mlruns")
# from mlflow.tracking import MlflowClient
# client = MlflowClient()

In [ ]:




# experiment = client.get_experiment_by_name("animal_classifier_v2")
# runs = client.search_runs([experiment.experiment_id])

# for run in runs:
#     print(f"Run ID: {run.info.run_id}")
#     print(f"Params: {run.data.params}")
#     print(f"Artifacts: {[a.path for a in client.list_artifacts(run.info.run_id)]}")
#     print("---")


In [ ]:
def evaluate_model_accuracy(model, dataloader, device="cpu"):
    model.to(device)
    model.eval()
    correct, total = 0, 0

    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            preds = torch.argmax(outputs, dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    return correct / total



def auto_evaluate_all_models(model_dir="../models", device="cpu"):
    
    _, _, test_loader = get_dataloaders()
    model_files = sorted(
        glob.glob(os.path.join(model_dir, "*.pt")),
        key=os.path.getctime
    )

    if not model_files:
        print("⚠️ No models found in folder.")
        return

    for path in model_files:
        model = get_model()
        model.load_state_dict(torch.load(path, map_location=device))
        acc = evaluate_model_accuracy(model, test_loader, device)

        timestamp = datetime.fromtimestamp(os.path.getctime(path)).strftime("%Y-%m-%d %H:%M:%S")
        print(f"📦 {os.path.basename(path)} — Accuracy: {acc:.4f} (Saved: {timestamp})")

auto_evaluate_all_models()



📦 resnet18_20250702_155546.pt — Accuracy: 0.9315 (Saved: 2025-07-02 15:56:47)
📦 resnet18_20250702_155758.pt — Accuracy: 0.9178 (Saved: 2025-07-02 16:00:24)
